In [1]:
import requests
url = "https://www.fantasypros.com/nba/stats/overall.php" # This will be the website source to crawl nba data table
response = requests.get(url)

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
    
html_string = '''
    <table>
        <tr>
            <td> Hello! </td>
            <td> Table </td>
        </tr>
    </table>
'''
    
soup = BeautifulSoup(html_string, 'lxml') # Parse the HTML as a string
    
table = soup.find_all('table')[0] # Grab the first table
    
new_table = pd.DataFrame(columns=range(0,2), index = [0]) # I know the size 
    
row_marker = 0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        new_table.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
    
new_table

,0,1
0,Hello!,Table


In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
    
class HTMLTableParser:
       
    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table['id'],self.parse_html_table(table))\
                for table in soup.find_all('table')]  
    
    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
        for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)
                        
                # Handle column names if we find them
            th_tags = row.find_all('th')                 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())
    
            # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")
    
        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                            index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1
                    
            # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass
            
        return df

In [4]:
hp = HTMLTableParser()
table = hp.parse_url(url)[0][1] # Grabbing the table from the tuple
table.head(3)

,Player,PTS,REB,AST,BLK,STL,FG%,FT%,3PM,TO,GP,MIN,FTM,2PM,A/TO,PF
0,"James Harden (HOU - PG,SG)","1,800",332.0,374.0,49.0,83.0,0.437,0.869,234.0,226.0,51.0,"1,891",532.0,283.0,1.65,174.0
1,Damian Lillard (POR - PG) OUT,"1,594",235.0,429.0,18.0,52.0,0.461,0.889,212.0,157.0,54.0,"1,996",368.0,295.0,2.73,91.0
2,Trae Young (ATL - PG),"1,464",215.0,452.0,5.0,56.0,0.445,0.856,173.0,237.0,49.0,"1,729",387.0,279.0,1.91,82.0


In [19]:
export_to_csv = table.to_csv (r'C:\Users\mylaptop\desktop\NBA_Data_export\export_NBA_dataframe.csv', index = None, header=True) # export to local machine as csv format